In [5]:
from collections import Counter, defaultdict
import numpy
import pandas
pandas.set_option('display.max_columns',200)
pandas.set_option('display.max_rows',200)

In [6]:
from wordle_tools import (
    SORTED_FIVE_LETTER_WORDS,
    ILLS_SPINE_WORDS,
    ILLS_STRONG_SPINES,
    map_df_back_to_strategy_dict,
    apply_wordle_strategy,
    min_max_groups_among_children,
    tabulate_strategy_from_search_results,
    wordl_score,
    top_one_off_patterns,
    filter_to_strong_spines,
    read_spine_file,
    calculate_max_group_size_given_guess
)

In [38]:
def play_from_strategy_dictionary_customised(stack_item, strategy_dict, root_word):
       
    if stack_item[1] == 0:
        return root_word

    
    strategy_dict_key = (
        stack_item[2],
        tuple(stack_item[3][1:])
    )
    
    # First run this function to build an initial tree
    # Then progressively add the custom rules after analysing the exceptions.
    
    if strategy_dict_key == (('LARNT',), ((1,0,0,0,0),)):
        return min_max_groups_among_children(
            stack_item,
            [
                'SCOWP', 'VUGHY', 'ZIMBS', 'VUGHS',
                'WOMBY', 'FJELD', 'VUGHY', 'WOMBS',
                'FUGLY', 'SPICK', 'VOZHD', 'WEMBS'
            ],
            'LARNT'
        ) # WEMBS is selected
    
    if strategy_dict_key == (('LARNT', 'WEMBS'),((1,0,0,0,0),(0,0,0,0,2))):
        return min_max_groups_among_children(
            stack_item,
            ['FUGLY', 'SPICK', 'VOZHD'],
            'LARNT'
        )
    
    if strategy_dict_key == (('LARNT', 'WEMBS', 'SPICK'),((1,0,0,0,0),(0,0,0,0,2), (1,0,1,0,0))):
        return min_max_groups_among_children(
            stack_item,
            ['FUGLY', 'VOZHD'],
            'LARNT'
        )
    if strategy_dict_key == (('LARNT', 'WEMBS'),((1,0,0,0,0),(0,2,0,0,2))):
        return "YIELD"    
    
    if strategy_dict_key == (('LARNT', 'WEMBS', 'YIELD'),((1,0,0,0,0),(0,2,0,0,2), (0,0,1,2,0))):
        return "SCHAV"    
    
    if strategy_dict_key == (('LARNT',), ((0,0,0,0,0),)):
        return "KEMBS"
    
    if strategy_dict_key == (('LARNT','KEMBS'), ((0,0,0,0,0),(0,2,0,0,2))):
        return "GIVED"
    
    if strategy_dict_key == (('LARNT','KEMPS'), ((0,2,0,0,0),(0,1,0,0,2))):
        return "GYVED"

    if strategy_dict_key == (('LARNT',), ((0,0,0,1,1),)):
        return "TINES"
    
    if strategy_dict_key == (('LARNT','TINES'), ((0,0,0,1,1),(1,0,2,1,2))):
        return "CHAWK"
    
    if strategy_dict_key == (('LARNT','TINES', 'CHAWK'), ((0,0,0,1,1),(1,0,2,1,2),(0,0,0,0,0))):
        #'LARNT', 'TINES', 'CHAWK', 'BADGE', 'SPIFF']
        return "BADGE" #then SPIFF but min-max will find it

    if strategy_dict_key == (('LARNT',), ((0,0,0,1,0),)):
        return "SINED"
    
    if strategy_dict_key == (('LARNT','SINED'), ((0,0,0,1,0),(1,2,2,2,0))):
        return "BACKS"
    
    if strategy_dict_key == (('LARNT','SINED', 'BACKS'), ((0,0,0,1,0),(1,2,2,2,0),(0,0,0,0,2))):
        return "FLIMP"
        
    if strategy_dict_key == (('LARNT',), ((0,0,1,0,0),)):
        return "IDEES"
    
    if strategy_dict_key == (('LARNT','IDEES'), ((0,0,1,0,0),(1,0,0,2,0))):
        return "VIBEX" # if (0,2,0,2,0) KEMPY
        
    if strategy_dict_key == (('LARNT','IDEES'), ((0,0,1,0,0),(0,0,2,1,0))):
        return "CABRE"
    
    if strategy_dict_key == (('LARNT',), ((0,0,0,0,1),)):
        return "SHOES"
    
    if strategy_dict_key == (('LARNT','SHOES'), ((0,0,0,0,1),(1,0,0,1,2))):
        return "TOWZY"
    
    if strategy_dict_key == (('LARNT','SHOES', 'TOWZY'), ((0,0,0,0,1),(1,0,0,1,2),(1,0,0,0,0))):
        return "KOPJE"  #then BEFOG but min-max will find it

    else:
        recorded_move = strategy_dict.get(
            strategy_dict_key,
            None
        )
    
        if recorded_move is None:

            print('missing move for:')
            print(stack_item)

            return min_max_groups_among_children(stack_item, SORTED_FIVE_LETTER_WORDS, 'LARNT')

        else:
            return recorded_move

In [64]:
df = pandas.read_csv(
    r'custom_tree_wordle_set_larnt_lte_6_CABRE_fix.csv',
).fillna("")

print(df['depth'].mean())
print(df['depth'].median())

Counter(df['depth'])

4.309744064138144
4.0


Counter({2: 30, 3: 1499, 1: 1, 4: 6311, 5: 4682, 6: 449})

In [65]:
strategy_dict = map_df_back_to_strategy_dict(df)

results, tree = apply_wordle_strategy(
    play_from_strategy_dictionary_customised, strategy_dict, 'LARNT'
)

In [66]:
list(strategy_dict.items())[334]

((('LARNT', 'TOTES'), ((2, 1, 0, 0, 1), (1, 0, 0, 1, 2))), 'LEATS')

In [67]:
Counter(r[1] for r in results)

Counter({2: 30, 3: 1499, 1: 1, 4: 6311, 5: 4682, 6: 449})

In [68]:
new_df = tabulate_strategy_from_search_results(results)
new_df[new_df['depth'] > 6]

,word,depth,g1,s11,s12,s13,s14,s15,g2,s21,s22,s23,s24,s25,g3,s31,s32,s33,s34,s35,g4,s41,s42,s43,s44,s45,g5,s51,s52,s53,s54,s55,g6,s61,s62,s63,s64,s65


In [69]:
new_df[new_df['depth'] > 6].groupby(list(new_df.columns)[1:8])['word'].count().reset_index()

,depth,g1,s11,s12,s13,s14,s15,word


In [70]:
new_df[new_df['depth'] > 6].groupby(list(new_df.columns)[1:14])['word'].count().reset_index()

,depth,g1,s11,s12,s13,s14,s15,g2,s21,s22,s23,s24,s25,word


In [71]:
new_df[new_df['depth'] > 6].groupby(list(new_df.columns)[1:20])['word'].count().reset_index()

,depth,g1,s11,s12,s13,s14,s15,g2,s21,s22,s23,s24,s25,g3,s31,s32,s33,s34,s35,word


In [72]:
new_df.to_csv('custom_tree_wordle_set_larnt_lte_6_CABRE_fix.csv', index=False)

In [73]:
words_1 = {
    w
    for w in SORTED_FIVE_LETTER_WORDS
    if wordl_score('LARNT', w) == (0,0,1,0,0)
    if wordl_score('IDEES', w) == (0,0,2,1,0)
    if wordl_score('CABRE', w) == (0, 0, 0, 2, 1)
}

len(words_1), words_1

(5, {'EMERG', 'EMERY', 'EVERY', 'PEERY', 'VEERY'})

In [74]:
Counter([wordl_score('CABRE', w) for w in words_1])

Counter({(0, 0, 0, 2, 1): 5})

In [75]:
words_1 = {
    w
    for w in SORTED_FIVE_LETTER_WORDS
    if wordl_score('LARNT', w) == (0,0,0,0,1)
    if wordl_score('SHOES', w) == (1,0,0,1,2)
    if wordl_score('TOWZY', w) == (1, 0, 0, 0, 0)
    if wordl_score('KOPJE', w) == (0, 0, 0, 0, 1)    
}

len(words_1), words_1

(4, {'BESTS', 'FESTS', 'GESTS', 'VESTS'})

In [76]:
sorted([
    (
        calculate_max_group_size_given_guess(guess, words_1),
        guess
    )
    for guess in SORTED_FIVE_LETTER_WORDS
])[:10]

[(1, 'BEFOG'),
 (2, 'ABAFT'),
 (2, 'ABOVE'),
 (2, 'AGAVE'),
 (2, 'AVGAS'),
 (2, 'BADGE'),
 (2, 'BAFFS'),
 (2, 'BAFFY'),
 (2, 'BAFTS'),
 (2, 'BAGEL')]

## Check the calculated tree is valid:

### Firstly check every input  has only one next guess:

In [77]:
input_cols = []
rule_cols = []

for n in range(1,6):
    input_cols += [f'g{n}',f's{n}1',f's{n}2',f's{n}3',f's{n}4',f's{n}5']
    rule_cols += input_cols + [f'g{n+1}']
    print(len(df[input_cols].drop_duplicates()) == len(df[rule_cols].drop_duplicates()))



True
True
True
True
True


### Check the word in the 'word' column is always in one of the guess 'g' columns:

In [78]:
Counter((
    df['g1'] == df['word']
) | (
    df['g2'] == df['word']
) | (
    df['g3'] == df['word']
) | (
    df['g4'] == df['word']
) | (
    df['g5'] == df['word']
) | (
    df['g6'] == df['word']
))

Counter({True: 12972})

### ... which is the same as the word count:

In [79]:
len(set(df['word']))

12972

### which is equal to the Wordle word count:

In [80]:
len(SORTED_FIVE_LETTER_WORDS)

12972

### Demonstrate that two critical six-paths isolate key off-by-one word sets: 

In [81]:
filter_to_strong_spines(
    [['LARNT', 'WEMBS', 'SPICK', 'VOZHD', 'FUGLY']],
    {w for w in SORTED_FIVE_LETTER_WORDS if w.endswith('ILLS')}
)

[['LARNT', 'WEMBS', 'SPICK', 'VOZHD', 'FUGLY']]

In [82]:
set(['LARNT', 'WEMBS', 'SPICK', 'VOZHD', 'FUGLY']) & set(SORTED_FIVE_LETTER_WORDS)

{'FUGLY', 'LARNT', 'SPICK', 'VOZHD', 'WEMBS'}

In [83]:
filter_to_strong_spines(
    [['LARNT', 'SHOES', 'TOWZY', 'KOPJE', 'BEFOG']],
    {w for w in SORTED_FIVE_LETTER_WORDS if w.endswith('ESTS')}
)

[['LARNT', 'SHOES', 'TOWZY', 'KOPJE', 'BEFOG']]

In [84]:
set(['LARNT', 'SHOES', 'TOWZY', 'KOPJE', 'BEFOG']) & set(SORTED_FIVE_LETTER_WORDS)

{'BEFOG', 'KOPJE', 'LARNT', 'SHOES', 'TOWZY'}

In [85]:
def output_tree_dataframe_to_plain_text(df, fp):
    
    lines = []
    
    for x in df.values:
        line = '' 
        for i, item in enumerate(x):
            if i > 1:
                if isinstance(item, str):
                    line += ' '
                    line += item.strip()
                    line += ' '
                elif i == 1:
                    line += str(int(item))
                    line += ' '
                elif isinstance(item, float):
                    line += str(int(item)) 
                elif isinstance(item, int):
                    line += str(int(item)) 

        line = line.replace('    ', ' ').replace('   ', ' ').replace('  ', ' ')
        lines.append(line)
        
    open(fp, 'w').write('\n'.join(sorted(lines)))

In [86]:
output_tree_dataframe_to_plain_text(
    df,
    'wordle_tree_larnt_lte_6_20220205.txt'
)